In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.0 MB/s eta 0:00:00


In [2]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")


Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
# 1

def segment_customers(customer_data):
    clustor_dict = {}
    for data in customer_data:
        age = data["age"]
        income = data["income"]
        interests = data["interests"]
        interests_ = " and ".join(interests)
        prompt = (
            f"Question: People with age {age}, income {income} and have interests in {interests_} belongs to which cluster from the following\n\n"
            "Clusters: Young, Affluent, Regular\n\nAnswer:"
        )

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
        )
        gen_text = tokenizer.batch_decode(gen_tokens)[0]
        prompt = "".join(prompt)
        gen_text = gen_text[len(prompt):]
        category = gen_text.split("\n\n")[0].strip()
        clustor_dict[category] = data
    return clustor_dict


customer_data = [{'age': 23, 'income': 55000, 'interests': ['fashion', 'tech']},
{'age': 35, 'income': 80000, 'interests': ['finance', 'travel']},
{'age': 50, 'income': 60000, 'interests': ['gardening', 'cooking']}]
customer_segments = segment_customers(customer_data)
print(customer_segments)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'Young': {'age': 50, 'income': 60000, 'interests': ['gardening', 'cooking']}}


In [6]:

# 2

def make_recommendations(customer, product_data):
    product_list = "\n".join([f"- product name: {product['name']}, product category: {product['category']}." for product in product_data])
    prompt = (
        f"Question: Recommend products for customer with age: {customer['age']} income: {customer['income']} and interests: {' and '.join(customer['interests'])} from the list below\n\nList:\n"
        f"{product_list}\n\nAnswer:"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    prompt = "".join(prompt)
    recommendations = gen_text[len(prompt):]
    return recommendations

customer = {'age': 23, 'income': 55000, 'interests': ['fashion', 'tech']}
product_data = [{'name': 'dress', 'category': 'fashion'},
{'name': 'laptop',
'category': 'tech'},
{'name': 'suit', 'category': 'fashion'},
{'name':
'desktop', 'category': 'tech'},
{'name': 'garden tools', 'category': 'gardening'}]
recommendations = make_recommendations(customer, product_data)
print(recommendations)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 This products are only for personal use


In [7]:
# 3

def gather_insights(customer_data):
    # gather insights about customers, such as their preferences, behaviors, and history with the company
    insights = {}
    for customer in customer_data:
        insights[customer['id']] = {'preferences': customer['interests'], 'purchase history':
        customer['purchases'], 'customer service interactions': customer['service interactions']}
    return insights

    
customer_data = [
{'id': 1, 'age': 23, 'income': 55000, 'interests': ['fashion', 'tech'],
'purchases': ['dress', 'laptop'], 'service interactions': ['phone call', 'email']},
{'id': 2, 'age': 35, 'income': 80000, 'interests': ['finance', 'travel'],
'purchases': ['suit', 'plane ticket'], 'service interactions': ['chat', 'email']},
{'id': 3, 'age': 50, 'income': 60000, 'interests': ['gardening', 'cooking'],
'purchases': ['garden tools', 'cookbook'], 'service interactions': ['phone call']}]
customer_insights = gather_insights(customer_data)
print(customer_insights)



{1: {'preferences': ['fashion', 'tech'], 'purchase history': ['dress', 'laptop'], 'customer service interactions': ['phone call', 'email']}, 2: {'preferences': ['finance', 'travel'], 'purchase history': ['suit', 'plane ticket'], 'customer service interactions': ['chat', 'email']}, 3: {'preferences': ['gardening', 'cooking'], 'purchase history': ['garden tools', 'cookbook'], 'customer service interactions': ['phone call']}}


In [8]:
# 4

def generate_leads(website_visitors):

    customer_list = "\n".join([f"- email: {customer['email']}, page views: {customer['page views']}, actions: {' and '.join(customer['actions'])}." for customer in website_visitors])
    prompt = (
        "Question: identify potential customers and return their emails as array from the list below\n\nList:\n"
        f"{customer_list}\n\nAnswer:"
    )

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    prompt = "".join(prompt)
    lead = gen_text[len(prompt):]
    return lead


website_visitors = [{'email': 'jane@example.com', 'page views': 10, 'actions': ['add to cart', 'contact']},
{'email': 'john@example.com', 'page views': 2, 'actions': ['add to cart']},
{'email': 'bob@example.com', 'page views': 7, 'actions': ['contact']}]
lead = generate_leads(website_visitors)
print(generate_leads(website_visitors))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 email

- jane@example.com, page views


In [4]:
# 5

def predict_customer_behavior(customer_data):
    customer_behavior = {}
    for idx, data in enumerate(customer_data):
        age = data["age"]
        income = data["income"]
        interests = data["interests"]
        interests_ = " and ".join(interests)
        prompt = (
            f"Question: People with age {age}, income {income} and have interests in {interests_} have the likely response to a product from the following responses\n\n"
            "Responses: likely to purchase, unlikely to purchase, neutral\n\nAnswer:"
        )

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
        )
        gen_text = tokenizer.batch_decode(gen_tokens)[0]
        prompt = "".join(prompt)
        gen_text = gen_text[len(prompt):]
        category = gen_text.split("\n\n")[0].strip()
        customer_behavior[idx] = category
    return customer_behavior


customer_data = [{'id': 1, 'age': 23, 'income': 55000, 'interests': ['fashion', 'tech']},
{'id': 2, 'age': 35, 'income': 80000, 'interests': ['finance', 'travel']},
{'id': 3, 'age': 50, 'income': 60000, 'interests': ['gardening', 'cooking']}]

customer_predictions = predict_customer_behavior(customer_data)
print(customer_predictions)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [9]:
# 6

def automate_marketing_tasks(tasks):
    generated_posts = []
    for task in tasks:
        if task['type'] == 'email':
             prompt = (
                f"Question: Write an email to {task['recipients']} about subject '{task['subject']}'"
                f" and convey message '{task['message']}'.\n\nAnswer:"
            )
        elif task['type'] == 'social media':
            prompt = (
                f"Question: Write social media post for {task['platform']}"
                f" to convey message '{task['message']}'.\n\nAnswer:"
            )
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
        )
        gen_text = tokenizer.batch_decode(gen_tokens)[0]
        prompt = "".join(prompt)
        gen_text = gen_text[len(prompt):]
        generated_posts.append(gen_text)
            
tasks = [{'type': 'email', 'recipients': ['customer1@example.com',
'customer2@example.com'], 'subject': 'New sale!', 'message': 'Check out our new sale items!'},
{'type': 'social media', 'platform': 'Facebook', 'message': 'Join us on Facebook to stay up to date with our latest products and promotions!'}]
generated_posts = automate_marketing_tasks(tasks)
print(generated_posts)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


None


In [12]:
# 7


def generate_content(content_type, target_audience):
    prompt = f"Question: Generate title name for {content_type} for audience target {target_audience}.\n\nTitle:"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    prompt = "".join(prompt)
    gen_text = gen_text[len(prompt):]
    print(gen_text)
    title = gen_text.split("\n\n")[0].strip()


    prompt = (
        f"Question: Write {content_type} for title {title}"
        f" for target audience {target_audience}.\n\nAnswer:"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    prompt = "".join(prompt)
    content = gen_text[len(prompt):]
    return {'title': title, 'content': content}
content_type = 'blog post'
target_audience = 'millennials'
content = generate_content(content_type, target_audience)
print(content)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Title Name:

Firstname Lastname

My name is Michael.

My title name is Michael.

My name is Kevin.

I wanna know what the audience's name is.

I wanna know what the audience's name is.

My name is Michael.

My title name is Michael.

My name is John.

My
{'title': '', 'content': ' Write blog post for title for target audience millennial.\n\nQuestion: Write blog post using blog post title as a starting point.\n\nAnswer: Write blog post using blog post title as a starting point.\n\nQuestion: How to add a custom tag?\n\nAnswer: If you want to edit the title, right-click on the title and choose "Add New" option on the shortcut menu,'}


In [ ]:
# 8
def manage_social_media(posts):
    generated_posts = []
    for post in posts:
        prompt = (
            f"Question: Write social media post for {post['platform']}"
            f" to convey message '{post['message']}'.\n\nAnswer:"
        )
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
        )
        gen_text = tokenizer.batch_decode(gen_tokens)[0]
        prompt = "".join(prompt)
        gen_text = gen_text[len(prompt):]
        generated_posts.append(gen_text)
    return generated_posts

posts = [{'platform': 'Facebook', 'message': 'Check out our latest product launch!'},
{'platform': 'Twitter', 'message': 'Exciting news! Our new product is nowavailable.'}]
posts = manage_social_media(posts)
print(posts)

In [ ]:
# 9

def colaborate_with_influencer(influencer_data, product_data):
    influencer_email = []
    for idx, data in enumerate(influencer_data):
        name = data["name"]
        email = data["email"]
        followers = data["followers"]
        interests = data["interests"]
        interests_ = " and ".join(interests)
        prompt = (
            f"Question: Write an email to influencer named {name}, having {followers} followers to email: {email} have interests in {interests_}"
            f" to colaborate on product named {product_data["name"]} that belongs to category {product_data["category"]}.\n\nAnswer:"
        )

        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        gen_tokens = model.generate(
            input_ids,
            do_sample=True,
            temperature=0.9,
            max_length=100,
        )
        gen_text = tokenizer.batch_decode(gen_tokens)[0]
        prompt = "".join(prompt)
        gen_email = gen_text[len(prompt):]
        influencer_email.append(gen_email)
    return influencer_email


influencer_data = [{'name': 'Bob', 'email': 'bob@example.com', 'followers': 50000,
'interests': ['tech', 'finance']}]
product_data = {'name': 'Lip balm', 'category': 'beauty'}
influencer_email = colaborate_with_influencer(influencer_data, product_data)
print(influencer_email)

In [ ]:
customer = {'age': 23, 'income': 55000, 'interests': ['fashion', 'tech']}
product_data = [{'name': 'dress', 'category': 'fashion'},
{'name': 'laptop',
'category': 'tech'},
{'name': 'suit', 'category': 'fashion'},
{'name':
'desktop', 'category': 'tech'},
{'name': 'garden tools', 'category': 'gardening'}]
recommendations = make_recommendations(customer, product_data)
print(recommendations) # [{'name': 'dress', 'category': 'fashion'},
#
{'name': 'laptop', 'category': 'tech'},
# {'name': 'suit', 'category':
'fashion'},
# {'name': 'desktop', 'category': 'tech'}]

In [ ]:
# 10

def target_ads(customer_data, ad_content):
    customer_list = "\n".join([f"- age: {customer['age']}, email: {customer['email']} and interests: {' and '.join(customer['interests'])}." for customer in customer_data])
    prompt = (
        f"Question: Whom should be targeted from the list below for the advertisement '{ad_content}' Return their emails.\n\nList:\n"
        f"{customer_list}\n\nAnswer:"
    )
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=100,
    )
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    prompt = "".join(prompt)
    customers = gen_text[len(prompt):]
    return customers

customer_data = [{'id': 1, 'age': 23, 'email': 'jane@example.com', 'interests':
['fashion', 'tech']},
{'id': 2, 'age': 35, 'email': 'john@example.com', 'interests': ['finance',
'travel']},
{'id': 3, 'age': 50, 'email': 'bob@example.com', 'interests': ['gardening',
'cooking']}]
ad_content = 'Check out our new tech product for young, trend-setting customers like you!'
customers = target_ads(customer_data, ad_content)
print(customers)